In [9]:
!pip install -U bitsandbytes

In [1]:
# imports

import os
import requests
from IPython.display import Markdown, display, update_display
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig
from dotenv import load_dotenv
import torch
import gradio as gr
import ollama

In [2]:
# Constants

LLAMA = "meta-llama/Meta-Llama-3.1-8B-Instruct"

In [3]:
# Sign in to HuggingFace Hub

load_dotenv()
hf_token = os.getenv('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [4]:
# System message

system_message = "Eres un asistente útil que generará datos sintéticos en base a una oración que será entregada por el usuario. "
system_message += "La modalidad es la siguiente: el usuario ingresa un prompt que inicia con la letra C de contexto y luego otro prompt con la letra D de datos a generar. "
system_message += "Por ejemplo: 'C:Deporte', tu respondes que estás preparado para generar datos sintéticos sobre deporte. El siguiente prompt será: 'D:Jugadores de fútbol de Liverpool' y "
system_message += "tu responderás con los datos."

In [14]:
# Initialize parameters

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
)
tokenizer = AutoTokenizer.from_pretrained(LLAMA)
tokenizer.pad_token = tokenizer.eos_token
streamer = TextStreamer(tokenizer)
model = AutoModelForCausalLM.from_pretrained(LLAMA, device_map="cpu", quantization_config=quant_config)

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [13]:
# Function chat

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt")
    outputs = model.generate(inputs, max_new_tokens=3000, streamer=streamer)
    return tokenizer.decode(outputs[0])

In [12]:
# Launch the interface

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/rodrigotristany/Desktop/Personal/llm-engineering-course/llm_engineering/llms/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rodrigotristany/Desktop/Personal/llm-engineering-course/llm_engineering/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rodrigotristany/Desktop/Personal/llm-engineering-course/llm_engineering/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2047, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rodrigotristany/Desktop/Personal/llm-engineering-course/llm_engineering/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1592, in call_function
    pred